In [29]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline
# Input data files are available in the "../input/" directory.

# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [30]:

train=pd.read_csv('../input/train_onehot.csv')
test=pd.read_csv('../input/test.csv')

train_dir='../input/train_scaled/train_scaled/'
test_dir='../input/test_scaled/test_scaled/'

train_images=np.array(train.iloc[:,0])
imagearr=[]
for i in train_images:
    img=Image.open(train_dir+i)
    imagearr.append(np.array(img))
X=np.array(imagearr)


In [31]:
test_images=np.array(test.iloc[:,0])
imagearr=[]
for i in test_images:
    img=Image.open(test_dir+i)
    imagearr.append(np.array(img))
Y=np.array(imagearr)

In [32]:
Y.shape

In [33]:

plt.imshow(X[0].reshape(64,64))

In [34]:
print(X[260].shape)
print(X.shape)
print(X.reshape(X.shape[0], 64, 64, 1).shape)
X=X.reshape(-1, 64, 64, 1)

In [35]:
print(X[0])
X=X/X.max()

In [36]:
Y=Y.reshape(-1, 64, 64, 1)

In [37]:
Y[0:20].shape

In [38]:

train.iloc[0,1:].values

In [39]:
X_labels=[]
for i in range(len(train)):
    X_labels.append(train.iloc[i,1:].values)


In [40]:
X_labels=np.array(X_labels)
X_labels

In [41]:
plt.imshow(X[5].reshape(64,64))
print(X_labels.shape)

In [42]:
from sklearn.model_selection import train_test_split
test_size = 0.3
X_train, X_test, Y_train, Y_test = train_test_split(X,X_labels, test_size=test_size, random_state=101)

img_size = 64
channel_size = 1
print("Training Size:", X_train.shape)
print(X_train.shape[0],"samples - ", X_train.shape[1],"x",X_train.shape[2],"grayscale image")

print("\n")

print("Test Size:",X_test.shape)
print(X_test.shape[0],"samples - ", X_test.shape[1],"x",X_test.shape[2],"grayscale image")

In [43]:
class SignClass():
    
    def __init__(self):
        self.i = 0
        
        self.training_images = X_train
        self.training_labels = Y_train
        
        self.test_images = X_test
        self.test_labels = Y_test
        self._epochs_completed = 0
        self._index_in_epoch = 0
        
        self._num_examples = X_train.shape[0]
    


        
    def next_batch(self, batch_size,fake_data=False, shuffle=True):
#         x = self.training_images[self.i:self.i+batch_size].reshape(-1,64,64,1)
#         y = self.training_labels[self.i:self.i+batch_size]
#         self.i = (self.i + batch_size) % len(self.training_images)
#         return x, y

        if fake_data:
            fake_image = [1] * 4096
            if self.one_hot:
                fake_label = [1] + [0] * 9
            else:
                fake_label = 0
            return [fake_image for _ in xrange(batch_size)], [fake_label for _ in xrange(batch_size)]
        start = self._index_in_epoch
        # Shuffle for the first epoch
        if self._epochs_completed == 0 and start == 0 and shuffle:
            perm0 = np.arange(self._num_examples)
            np.random.shuffle(perm0)
            self.training_images = self.training_images[perm0]
            self.training_labels = self.training_labels[perm0]
        # Go to the next epoch
        if start + batch_size > self._num_examples:
        # Finished epoch
            self._epochs_completed += 1
            # Get the rest examples in this epoch
            rest_num_examples = self._num_examples - start
            images_rest_part = self.training_images[start:self._num_examples]
            labels_rest_part = self.training_labels[start:self._num_examples]
          # Shuffle the data
            if shuffle:
                perm = np.arange(self._num_examples)
                np.random.shuffle(perm)
                self.training_images = self.training_images[perm]
                self.training_labels = self.training_labels[perm]
            # Start next epoch
            start = 0
            self._index_in_epoch = batch_size - rest_num_examples
            end = self._index_in_epoch
            images_new_part = self.training_images[start:end]
            labels_new_part = self.training_labels[start:end]
            return np.concatenate((images_rest_part, images_new_part), axis=0), np.concatenate((labels_rest_part, labels_new_part), axis=0)
        else:
            self._index_in_epoch += batch_size
            end = self._index_in_epoch
            return self.training_images[start:end], self.training_labels[start:end]

In [44]:
ch = SignClass()

In [45]:
import tensorflow as tf

In [46]:
x = tf.placeholder(tf.float32,shape=[None,64,64,1],name="x")
y_true = tf.placeholder(tf.float32,shape=[None,30],name="y_true")

In [47]:
hold_prob = tf.placeholder(tf.float32,name="hold_prob")
tf.summary.scalar('dropout_keep_probability', hold_prob)

In [48]:
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(init_random_dist,name="W")

def init_bias(shape):
    init_bias_vals = tf.constant(0.1, shape=shape)
    return tf.Variable(init_bias_vals,name="b")

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2by2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')

def convolutional_layer(input_x, shape,name="conv_layer"):
    with tf.name_scope(name):
        W = init_weights(shape)
        b = init_bias([shape[3]])
        c=conv2d(input_x, W)
        act=tf.nn.relu(c + b)
        tf.summary.histogram("weights",W)
        tf.summary.histogram("biases",b)
        tf.summary.histogram("activations",act)
        return act

def normal_full_layer(input_layer, size,name="normal_layer"):
    with tf.name_scope(name):
        input_size = int(input_layer.get_shape()[1])
        W = init_weights([input_size, size])
        b = init_bias([size])
        return tf.matmul(input_layer, W) + b

In [49]:
convo_1 = convolutional_layer(x,shape=[4,4,1,64])
convo_1_pooling = max_pool_2by2(convo_1)

In [50]:
convo_2 = convolutional_layer(convo_1_pooling,shape=[4,4,64,128])
convo_2_pooling = max_pool_2by2(convo_2)

In [51]:
convo_2_flat = tf.reshape(convo_2_pooling,[-1,16*16*128])
full_layer_one = tf.nn.relu(normal_full_layer(convo_2_flat,1024))

In [52]:
full_one_dropout = tf.nn.dropout(full_layer_one,keep_prob=hold_prob)

In [53]:
y_pred = normal_full_layer(full_one_dropout,30)

In [54]:
with tf.name_scope("crossentropy"):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_true,logits=y_pred))
    tf.summary.scalar('cross_entropy', cross_entropy)
with tf.name_scope("optimizer"):
    optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
    train = optimizer.minimize(cross_entropy)
init = tf.global_variables_initializer()


In [55]:
steps = 500
saver = tf.train.Saver()
with tf.Session() as sess:
    
    sess.run(init)
    merged_Summary=tf.summary.merge_all()
    writer=tf.summary.FileWriter('dir3/')
    writer.add_graph(sess.graph)
    for i in range(steps):
        batch = ch.next_batch(1000)
        
        sess.run(train, feed_dict={x: batch[0], y_true: batch[1], hold_prob: 0.5})
        
        # PRINT OUT A MESSAGE EVERY 100 STEPS
        if i%100 == 0:
            s=sess.run(merged_Summary,feed_dict={x: batch[0], y_true: batch[1], hold_prob: 0.5})
            writer.add_summary(s,i)
            print('Currently on step {}'.format(i))
            print('Accuracy is:')
            # Test the Train Model
            with tf.name_scope("accuracy"):
                matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))

                acc = tf.reduce_mean(tf.cast(matches,tf.float32))
                tf.summary.scalar('accuracy', acc)
                print(sess.run(acc,feed_dict={x:ch.test_images,y_true:ch.test_labels,hold_prob:1.0}))
                print('\n')
    save_path = saver.save(sess, "/tmp/model.ckpt")
    print("Model saved in path: %s" % save_path)
    

In [ ]:
#import tensorflow as tf
predictions=[]
with tf.Session() as sess:
  # Restore variables from disk.
    saver.restore(sess, "/tmp/model.ckpt")
    print('predicting')
    predicta=tf.argmax(y_pred,1)
    predicts=tf.nn.softmax(y_pred)
    print(1)
#     print(sess.run(tf.Print(y_pred,[y_pred])))
#     print(sess.run(tf.Print(predicta,[predicta])))
#     print(sess.run(tf.Print(predicts,[predicts])))
    
    predictions1=sess.run(y_pred,feed_dict={x:Y[0:1000],hold_prob:1.0})
    predictions2=sess.run(y_pred,feed_dict={x:Y[1000:2000],hold_prob:1.0})
    predictions3=sess.run(y_pred,feed_dict={x:Y[2000:3000],hold_prob:1.0})
    predictions4=sess.run(y_pred,feed_dict={x:Y[3000:4000],hold_prob:1.0})
    predictions5=sess.run(y_pred,feed_dict={x:Y[4000:5000],hold_prob:1.0})
    predictions6=sess.run(y_pred,feed_dict={x:Y[5000:],hold_prob:1.0})

# for j in [predictions1,predictions2,predictions3,predictions4,predictions5,predictions6]:
    
#     predictions.append(j)
#predictions=predictions1+predictions2+predictions3+predictions4+predictions5+predictions6
# for j in predictions1:
#     predictions.append([format(float(x), '.16f') for x in j])
# for j in predictions2:
#     predictions.append([format(float(x), '.16f') for x in j])
# for j in predictions3:
#     predictions.append([format(float(x), '.16f') for x in j])
# for j in predictions4:
#     predictions.append([format(float(x), '.16f') for x in j])
# for j in predictions5:
#     predictions.append([format(float(x), '.16f') for x in j])
# for j in predictions6:
#     predictions.append([format(float(x), '.16f') for x in j])

for j in predictions1:
    predictions.append([format(float(x), '.16f') for x in j])
for j in predictions2:
    predictions.append([format(float(x), '.16f') for x in j])
for j in predictions3:
    predictions.append([format(float(x), '.16f') for x in j])
for j in predictions4:
    predictions.append([format(float(x), '.16f') for x in j])
for j in predictions5:
    predictions.append([format(float(x), '.16f') for x in j])
for j in predictions6:
    predictions.append([format(float(x), '.16f') for x in j])
predictions=np.array(predictions)
print('done >',predictions.shape)

In [ ]:
predictions.shape

In [ ]:
trainmain=pd.read_csv('../input/train_onehot.csv')
trainmain.drop('Image_id',1).columns

In [ ]:
pr=pd.DataFrame(data=predictions,columns=trainmain.drop('Image_id',1).columns)
mmm=pd.concat([pd.DataFrame(data=test_images,columns=['image_id']),pr],axis=1)


In [ ]:
mmm.head()

In [ ]:
mmm.to_csv('pr11.csv',index=False)

In [ ]:
mmm.iloc[1,:]

In [ ]:
mmm.head()

In [ ]:
predictions1.astype(float)